In [ ]:
# COLLECTING DATA

In [ ]:
!pip install pandas requests beautifulsoup4 scikit-learn matplotlib seaborn plotly

In [7]:
# Collect player salary

import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL to salary table
url = 'https://www.basketball-reference.com/contracts/players.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the salary table
table = soup.find('table', {'id': 'player-contracts'})
df_salary_raw = pd.read_html(str(table))[0]

df_salary_raw.head()

/tmp/ipython-input-7-1116387556.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_salary_raw = pd.read_html(str(table))[0]


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0       Salary  \
                  Rk             Player                 Tm      2024-25   
0                  1      Stephen Curry                GSW  $55,761,216   
1                  2        Joel Embiid                PHI  $51,415,938   
2                  3      Nikola JokiÄ                DEN  $51,415,938   
3                  4       Kevin Durant                PHO  $51,179,021   
4                  5       Bradley Beal                PHO  $50,203,930   

                                                               \
       2025-26      2026-27      2027-28      2028-29 2029-30   
0  $59,606,817  $62,587,158          NaN          NaN     NaN   
1  $55,224,526  $57,985,752  $62,624,612  $67,263,472     NaN   
2  $55,224,526  $59,033,114  $62,841,702          NaN     NaN   
3  $54,708,609          NaN          NaN          NaN     NaN   
4  $53,666,270  $57,128,610          NaN          NaN     NaN   

  Unnamed: 9_level_0  
          Guaranteed  
0       $177,955,191  
1       $227,250,828  
2       $165,673,578  
3       $105,887,630  
4       $103,870,200

In [9]:
# Collect/Merge player stats

# Function to clean player names
def clean_player_names(df, player_col='Player'):
    df[player_col] = df[player_col].str.replace('*', '', regex=False)
    df[player_col] = df[player_col].str.strip()
    return df

# URLs for scraping
url_basic = "https://www.basketball-reference.com/leagues/NBA_2024_per_game.html"
url_advanced = "https://www.basketball-reference.com/leagues/NBA_2024_advanced.html"

# --- Scrape Per-Game Stats ---
tables_basic = pd.read_html(url_basic)
df_basic_raw = tables_basic[0]

# Drop repeated headers inside the table
df_basic_raw = df_basic_raw[df_basic_raw['Player'] != 'Player'].reset_index(drop=True)

# Clean player names in Basic stats data
df_basic_raw = clean_player_names(df_basic_raw)

# --- Scrape Advanced Stats ---
tables_adv = pd.read_html(url_advanced)
df_adv_raw = tables_adv[0]

# Drop repeated headers inside the table
df_adv_raw = df_adv_raw[df_adv_raw['Player'] != 'Player'].reset_index(drop=True)

# Clean player names in Advanced data
df_adv_raw = clean_player_names(df_adv_raw)

# Merge DataFrames on Player and Team
df_stats_raw = pd.merge(
    df_basic_raw,
    df_adv_raw,
    how='inner',
    on=['Player', 'Team'],
    suffixes=('_per', '_adv')
)

# Show info and preview
print(f"Merged dataframe shape: {df_stats_raw.shape}")
df_stats_raw.head(15)

Merged dataframe shape: (736, 58)


,Rk_per,Player,Age_per,Team,Pos_per,G_per,GS_per,MP_per,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards_adv
0,1.0,Joel Embiid,29.0,PHI,C,39.0,39.0,33.6,11.5,21.8,...,39.6,5.2,2.3,7.5,0.275,8.5,3.1,11.6,4.5,AS
1,2.0,Luka Dončić,24.0,DAL,PG,70.0,70.0,37.5,11.5,23.6,...,36.0,8.5,3.5,12.0,0.220,8.3,1.7,9.9,8.0,"MVP-3,CPOY-6,AS,NBA1"
2,3.0,Giannis Antetokounmpo,29.0,MIL,PF,73.0,73.0,35.2,11.5,18.8,...,33.0,9.5,3.7,13.2,0.246,6.7,2.4,9.0,7.2,"MVP-4,DPOY-9,CPOY-12,AS,NBA1"
3,4.0,Shai Gilgeous-Alexander,25.0,OKC,PG,75.0,75.0,34.0,10.6,19.8,...,32.8,10.5,4.2,14.6,0.275,6.7,2.3,9.0,7.1,"MVP-2,DPOY-7,CPOY-3,AS,NBA1"
4,5.0,Jalen Brunson,27.0,NYK,PG,77.0,77.0,35.4,10.3,21.4,...,32.5,8.8,2.4,11.2,0.198,6.3,-0.4,5.8,5.4,"MVP-5,CPOY-5,AS,NBA2"
5,6.0,Devin Booker,27.0,PHO,PG,68.0,68.0,36.0,9.4,19.2,...,29.8,7.2,2.0,9.2,0.181,4.0,-1.0,3.0,3.1,"AS,NBA3"
6,7.0,Kevin Durant,35.0,PHO,PF,75.0,75.0,37.2,10.0,19.1,...,29.0,5.1,3.2,8.3,0.142,4.0,0.1,4.0,4.3,"MVP-9,AS,NBA2"
7,8.0,Jayson Tatum,25.0,BOS,PF,74.0,74.0,35.7,9.1,19.3,...,30.2,6.4,4.1,10.4,0.189,4.5,0.6,5.1,4.7,"MVP-6,CPOY-9,AS,NBA1"
8,9.0,De'Aaron Fox,26.0,SAC,PG,74.0,74.0,35.9,9.7,20.9,...,31.0,3.3,3.2,6.5,0.117,2.6,0.1,2.7,3.2,CPOY-12
9,10.0,Donovan Mitchell,27.0,CLE,SG,55.0,55.0,35.3,9.1,19.8,...,31.4,3.6,2.9,6.4,0.159,4.6,1.2,5.8,3.9,AS


In [17]:
# Download raw data to CSV (cannot write to Excel due to multi-index)

df_stats_raw.to_csv("stats_raw.csv", index=False)
df_salary_raw.to_csv("salaries_raw.csv", index=False)

from google.colab import files
files.download("stats_raw.csv")

from google.colab import files
files.download("salaries_raw.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# CLEANING DATA STEP 1

In [19]:
# Clean player salary table

def clean_salary_column(df, column=""):
    df[column] = (
        df[column]
        .astype(str)
        .str.replace("\n", "", regex=False)       # Handle line breaks
        .str.replace("_", " ", regex=False)
        .str.encode("latin1")
        .str.decode("utf-8")
        .str.strip()
        .str.title()
    )
    return df

# Flatten MultiIndex columns: cannot use .str (below) if multiindex
if isinstance(df_salary_raw.columns, pd.MultiIndex):
    df_salary_raw.columns = ["_".join(col).strip() for col in df_salary_raw.columns.values]

# Rename headers
df_salary_raw.rename(columns={"Unnamed: 1_level_0_Player": "player", "Unnamed: 2_level_0_Tm": "team", "Salary_2024-25": "salary_2024_25"}, inplace=True)

# Remove whitespace and unwanted characters from all column headers only
df_salary_raw.columns = df_salary_raw.columns.str.replace("-","_").str.replace("\n","").str.strip().str.lower()

# clean specific column values via cleaning function
df_salary_raw = clean_salary_column(df_salary_raw, "player")
df_salary_raw = clean_salary_column(df_salary_raw, "team")

# Remove $ and commas, then convert to float
df_salary_raw["salary_2024_25"] = (df_salary_raw["salary_2024_25"].str.replace("$", "", regex=False).str.replace(",", "", regex=False))
df_salary_raw = df_salary_raw[df_salary_raw["salary_2024_25"].str.replace(" ", "").str.isnumeric()]
df_salary_raw["salary_2024_25"] = df_salary_raw["salary_2024_25"].astype(float)

df_salary_raw.head(20)

/tmp/ipython-input-19-3629719927.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salary_raw["salary_2024_25"] = df_salary_raw["salary_2024_25"].astype(float)


,unnamed: 0_level_0_rk,player,team,salary_2024_25,salary_2025_26,salary_2026_27,salary_2027_28,salary_2028_29,salary_2029_30,unnamed: 9_level_0_guaranteed
0,1,Stephen Curry,Gsw,55761216.0,"$59,606,817","$62,587,158",NaN,NaN,NaN,"$177,955,191"
1,2,Joel Embiid,Phi,51415938.0,"$55,224,526","$57,985,752","$62,624,612","$67,263,472",NaN,"$227,250,828"
2,3,Nikola Jokić,Den,51415938.0,"$55,224,526","$59,033,114","$62,841,702",NaN,NaN,"$165,673,578"
3,4,Kevin Durant,Pho,51179021.0,"$54,708,609",NaN,NaN,NaN,NaN,"$105,887,630"
4,5,Bradley Beal,Pho,50203930.0,"$53,666,270","$57,128,610",NaN,NaN,NaN,"$103,870,200"
5,6,Jaylen Brown,Bos,49205800.0,"$53,142,264","$57,078,728","$61,015,192","$64,951,656",NaN,"$285,393,640"
6,7,Devin Booker,Pho,49205800.0,"$53,142,264","$57,078,728","$61,015,192",NaN,NaN,"$220,441,984"
7,8,Karl-Anthony Towns,Nyk,49205800.0,"$53,142,264","$57,078,728","$61,015,192",NaN,NaN,"$159,426,792"
8,9,Paul George,Phi,49205800.0,"$51,666,090","$54,126,380","$56,586,670",NaN,NaN,"$154,998,270"
9,10,Kawhi Leonard,Lac,49205800.0,"$50,000,000","$50,300,000",NaN,NaN,NaN,"$149,505,800"


In [20]:
# Clean player stats table


# Clean column headers
def clean_column_names(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace('\n', '')
        .str.replace(r'[^\w\s]', '', regex=True)
    )
    return df

df_stats_raw = clean_column_names(df_stats_raw)

# clean entire column function (excluding the header)
def clean_stats_column(df, column=""):
    df[column] = (
        df[column]
        .astype(str)
        .str.replace("\n", "", regex=False)       # Handle line breaks
        .str.replace("_", " ", regex=False)
        .str.strip()
        .str.title()
    )
    return df

# cleaning specific columns
df_stats_raw = clean_stats_column(df_stats_raw, "player")
df_stats_raw = clean_stats_column(df_stats_raw, "team")
df_stats_raw.columns = df_stats_raw.columns.str.replace('_per', '', regex=False)

df_stats_raw.head()

,rk,player,age,team,pos,g,gs,mp,fg,fga,...,usg,ows,dws,ws,ws48,obpm,dbpm,bpm,vorp,awards_adv
0,1.0,Joel Embiid,29.0,Phi,C,39.0,39.0,33.6,11.5,21.8,...,39.6,5.2,2.3,7.5,0.275,8.5,3.1,11.6,4.5,AS
1,2.0,Luka Dončić,24.0,Dal,PG,70.0,70.0,37.5,11.5,23.6,...,36.0,8.5,3.5,12.0,0.220,8.3,1.7,9.9,8.0,"MVP-3,CPOY-6,AS,NBA1"
2,3.0,Giannis Antetokounmpo,29.0,Mil,PF,73.0,73.0,35.2,11.5,18.8,...,33.0,9.5,3.7,13.2,0.246,6.7,2.4,9.0,7.2,"MVP-4,DPOY-9,CPOY-12,AS,NBA1"
3,4.0,Shai Gilgeous-Alexander,25.0,Okc,PG,75.0,75.0,34.0,10.6,19.8,...,32.8,10.5,4.2,14.6,0.275,6.7,2.3,9.0,7.1,"MVP-2,DPOY-7,CPOY-3,AS,NBA1"
4,5.0,Jalen Brunson,27.0,Nyk,PG,77.0,77.0,35.4,10.3,21.4,...,32.5,8.8,2.4,11.2,0.198,6.3,-0.4,5.8,5.4,"MVP-5,CPOY-5,AS,NBA2"


In [21]:
# Remove duplicates

df_stats = df_stats_raw.drop_duplicates(subset=['player', 'team'])
df_salary = df_salary_raw.drop_duplicates(subset=['player', 'team'])

In [22]:
# Merge both datasets into 1

salary_subset = df_salary[['player', 'team', 'salary_2024_25']]
df_merged = pd.merge(df_stats, salary_subset, on=['player'], how='left')

df_merged.head()

,rk,player,age,team_x,pos,g,gs,mp,fg,fga,...,dws,ws,ws48,obpm,dbpm,bpm,vorp,awards_adv,team_y,salary_2024_25
0,1.0,Joel Embiid,29.0,Phi,C,39.0,39.0,33.6,11.5,21.8,...,2.3,7.5,0.275,8.5,3.1,11.6,4.5,AS,Phi,51415938.0
1,2.0,Luka Dončić,24.0,Dal,PG,70.0,70.0,37.5,11.5,23.6,...,3.5,12.0,0.220,8.3,1.7,9.9,8.0,"MVP-3,CPOY-6,AS,NBA1",Lal,43031940.0
2,3.0,Giannis Antetokounmpo,29.0,Mil,PF,73.0,73.0,35.2,11.5,18.8,...,3.7,13.2,0.246,6.7,2.4,9.0,7.2,"MVP-4,DPOY-9,CPOY-12,AS,NBA1",Mil,48787676.0
3,4.0,Shai Gilgeous-Alexander,25.0,Okc,PG,75.0,75.0,34.0,10.6,19.8,...,4.2,14.6,0.275,6.7,2.3,9.0,7.1,"MVP-2,DPOY-7,CPOY-3,AS,NBA1",Okc,35859950.0
4,5.0,Jalen Brunson,27.0,Nyk,PG,77.0,77.0,35.4,10.3,21.4,...,2.4,11.2,0.198,6.3,-0.4,5.8,5.4,"MVP-5,CPOY-5,AS,NBA2",Nyk,24960001.0


In [23]:
# Store/Download partially cleaned data to Excel

df_merged.to_excel("dataset_partial_cleaned.xlsx", index=False)

from google.colab import files
files.download("dataset_partial_cleaned.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>